### Proyecto Final
Actualmente el proyecto realiza lo siguiente: 
- Genera la lista de canciones mediante one shoot prompt en gpt.
- Genera y almacena la imagen para la portada de la playlist mediante zero shoot prompt con dall-e
- Busca con la API de spotify la lista de canciones recomendadas por la IA
- Crea la lista de canciones con aquellas que fueron encontradas
- Setea el titulo de la playlist acorde al tema elegido.

In [57]:
# Library imports
import json
import requests
import openai
import os 
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import base64
from PIL import Image
from dotenv import dotenv_values

In [58]:
# Env variables 
# Para poder probar el código es necesario cargar las credenciales de la API de OpenAI y Spotify en un archivo .env
config = dotenv_values(".env")
openai.api_key = openai.api_key = config["OPENAI_API_KEY"]
SPOTIFY_CLIENT_ID = config["SPOTIFY_CLIENT_ID"]
SPOTIFY_CLIENT_SECRET = config["SPOTIFY_CLIENT_SECRET"]

In [59]:
# get sound list
def get_playlist(topic):

    prompt = f"""
        Necesito que me proporciones canciones infantiles cuya tematica sea: {topic} 
        Este contenido debe encontrarse en español, no debe repetirse
        Enfócate principalmente en que la canción contenga la temática que se quiere enseñar al niño.
        """


    context = """Eres un asistente útil para generar listas de reproducción.
                Deberías generar una lista de canciones y sus artistas según la indicación que se solicita.
                Debes ser claro, exacto y basarte en la información que contengas almacenada.
                Deberás devolver un array JSON, donde cada elemento sigue este formato:
                {"song": <song_title>, "artist": <artist_name>}
                """

    content = f"Generar una lista de 5 canciones basadas en este prompt: {prompt}"
    
    messages = [
        { "role": "system", "content": context},
        { "role": "user", "content": content, }
    ]

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages)

    playlist = json.loads(response.choices[0].message.content)
    return playlist

In [60]:
# Generate image
def generate_image(topic):
    prompt = f"""
    Necesito que generes una ilustración para niños, donde se muestre en el centro de la imagen a un niño escuchando música con 
    sus auriculares, lleno de lápices y libros y alrededor de su imagen, que muestre la temática de lo que está escuchando, 
    en este caso, el aprendizaje del {topic}. Utiliza colores cálidos e ilustraciones amigables para niños
    """

    # Make the request for the image prompt
    image_response = openai.Image.create(
        prompt=prompt,
        n=1,
        size="512x512"
    )
   
    return image_response['data'][0]['url']


In [61]:
def store_image(topic, image_url):
    # Download image
    response = requests.get(image_url)

    # Verify if possible to download
    if response.status_code == 200:
        # Get filename
        filename = os.path.basename(topic)
        # Agregar la extensión ".png"
        filename += ".jpg"
    
        # Directory to save the image
        directory = "./generated_images/"
        
        # Make directory if not exists
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        # Complete path
        filepath = os.path.join(directory, filename)
        
        # Save the image
        with open(filepath, 'wb') as f:
            f.write(response.content)
            
        return filepath

    else:
        print("Hubo un error al descargar la imagen.")

In [62]:
# Resize image
def image_resize(path): 
    # Abrir la imagen descargada
    img = Image.open(path)

    # Redimensionar la imagen a un tamaño específico (por ejemplo, 300x300 píxeles)
    img = img.resize((300, 300))

    # Guardar la imagen comprimida en un formato JPEG con una calidad específica (por ejemplo, calidad 80)
    img.save(path, optimize=True, quality=80)

In [63]:
# Create playlist
def add_songs_to_spotify(topic, playlist, path):

    sp = spotipy.Spotify( auth_manager=spotipy.SpotifyOAuth( client_id= SPOTIFY_CLIENT_ID, 
                                                            client_secret= SPOTIFY_CLIENT_SECRET, 
                                                            redirect_uri="http://localhost:9999", 
                                                            scope="playlist-modify-private"))

    current_user = sp.current_user()

    assert current_user is not None

    track_uris = []
    for item in playlist:
        artist, song = item["artist"], item["song"]

        advanced_query = f"artist:({artist}) track:({song})"
        basic_query = f"{song} {artist}"

        for query in [advanced_query, basic_query]:
            search_results = sp.search(
                q=query, limit=10, type="track"
            )

            if (
                not search_results["tracks"]["items"]
                or search_results["tracks"]["items"][0]["popularity"] < 20
            ):
                continue
            else:
                good_guess = search_results["tracks"]["items"][0]
                print(f"Encontrada: {good_guess['name']} [{good_guess['id']}]")
                track_uris.append(good_guess["id"])
                break

        else:
            print(
                f"Consultas {advanced_query} y {basic_query} no devuelve buenos resultados. Ignorando."
            )

    created_playlist = sp.user_playlist_create(
        current_user["id"],
        public=False,
        name=f"{topic}",
    )
    playlistID = created_playlist["id"]

    # Añadir canciones a la lista de reproducción
    sp.user_playlist_add_tracks(current_user["id"], created_playlist["id"], track_uris)

    
    # Cargar la imagen de la portada de la lista de reproducción
    with open(path, 'rb') as image_file:
        image_resize(path)
        image_64_encode = base64.b64encode(image_file.read())
        if len(image_64_encode) > 256000: # check if image is too big
            print("Image is too big: ", len(image_file.read()))
        else:
            sp.playlist_upload_cover_image(playlistID, image_64_encode)
            print("Image added.")

    
    print("\n")
    print(f"Lista de reproducción creada: {created_playlist['name']}")
    print(created_playlist["external_urls"]["spotify"])

In [ ]:
# Inputs
topic = input("Seleccione el tema a tratar: ") 
print(f"Creando playlist del tema elegido [{topic}]") # Ejemplo: El abecedario
playlist = get_playlist(topic)
image_url = generate_image(topic)
path = store_image(topic, image_url)
add_songs_to_spotify(topic, playlist, path)